In [ ]:
import time
from pinpong.board import Board
from pinpong.extension.unihiker import accelerometer, gyroscope
from unihiker import GUI
import socket
import math

Board().begin()
gui = GUI()
UDP_IP1 = "192.168.137.49"  # ESP32S3 IP address
UDP_PORT1 = 4210  # ESP32S3 port
UDP_IP2 = "192.168.137.115"  # ESP8266 IP address
UDP_PORT2 = 4212#ESP8266 PORT
UDP_IP3 = "192.168.137.13"  # ESP8266 home automation IP address
UDP_PORT3 = 4215#ESP8266 home automation PORT
sock = socket.socket(socket.AF_INET, socket.SOCK_DGRAM)

# Fall detection thresholds 
ACCEL_THRESHOLD = 1.0  
GYRO_THRESHOLD = 0.3 

Fan_state = False
light_state = False
WC_state = False

def detect_motion():
    while True:
        acc_x = accelerometer.get_x()
        acc_y = accelerometer.get_y()
        acc_z = accelerometer.get_z()
        gyr_x = gyroscope.get_x()
        gyr_y = gyroscope.get_y()
        gyr_z = gyroscope.get_z()

        acc_magnitude = math.sqrt(acc_x**2 + acc_y**2 + acc_z**2)
        gyro_magnitude = math.sqrt(gyr_x**2 + gyr_y**2 + gyr_z**2)

        if acc_magnitude > ACCEL_THRESHOLD and gyro_magnitude > GYRO_THRESHOLD:
            fall_alert = "FD"
            sock.sendto(fall_alert.encode(), (UDP_IP2, UDP_PORT2))
            print("Fall detected! Sending alert...")

        data_string = f"AX:{acc_x:.2f},AY:{acc_y:.2f},AZ:{acc_z:.2f},GX:{gyr_x:.2f},GY:{gyr_y:.2f},GZ:{gyr_z:.2f}"
        sock.sendto(data_string.encode(), (UDP_IP1, UDP_PORT1))

        gui.fill_rect(x=0, y=0, w=240, h=500, color="pink")
        gui.add_button(x=20, y=20, w=200, h=50, text="SOS", onclick=SOS)
        gui.add_button(x=20, y=80, w=200, h=50, text=f"Light {State(light_state)}", onclick=toggle_light)
        gui.add_button(x=20, y=140, w=200, h=50, text=WheelChair(WC_state), onclick=toggle_mode)
        gui.add_button(x=20, y=200, w=200, h=50, text=f"Fan {State(Fan_state)}",onclick=toggle_fan)
        time.sleep(0.5) 

def SOS():
    fall_alert = "SOS"
    sock.sendto(fall_alert.encode(), (UDP_IP2, UDP_PORT2))
    print("SOS DETECTED")

def toggle_light():
    global light_state
    if light_state:
        print('Light Off')
        data_string = 'loff'
    else:
        print('Light On')
        data_string = 'lon'
    
    sock.sendto(data_string.encode(), (UDP_IP3, UDP_PORT3))
    light_state = not light_state
def toggle_fan():
    global Fan_state
    if Fan_state:
        print('Fan Off')
        data_string = 'foff'
    else:
        print('Fan On')
        data_string = 'fon'
    
    sock.sendto(data_string.encode(), (UDP_IP3, UDP_PORT3))
    Fan_state = not Fan_state
def toggle_mode():
    global WC_state
    if WC_state:
        print('Back to wheel chair')
        data_string = 'WC'
    else:
        print('Wheel Chair is stretching')
        data_string = 'S'
    
    sock.sendto(data_string.encode(), (UDP_IP2, UDP_PORT2))
    WC_state = not WC_state

def State(a):
    return "on" if a else "off"

def WheelChair(a):
    return "Stretch" if a else "WheelChair"

detect_motion()
